aniiyer@iu.edu
CCL 

In [45]:
import numpy as np
import pandas as pd
from os import mkdir, getcwd, chdir, listdir
from os.path import join
import nltk
#nltk.download('punkt')
from textwrap import wrap

In [10]:
# # !rm -rf bert
# # !git clone https://github.com/google-research/bert
# import sys
# sys.path.append('bert/')
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import codecs
# import collections
# import json
# import re
# import os
# import pprint
# import numpy as np
# import tensorflow as tf
# import modeling
# import tokenization

In [2]:
pip install translate-api

  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14588 sha256=5c71625de677ec29cfed02a29191036b1be645a3bdc77b1cb02e5454e9287a73
  Stored in directory: c:\users\anirudh\appdata\local\pip\cache\wheels\db\3c\3d\7e9aca234caf6602ae4a4c7b367b3afc03519e791b998a94e4
Successfully built PyExecJS
Note: you may need to restart the kernel to use updated packages.


In [51]:
embedded_dictionary = {}
DIRECTORY = getcwd()
with open(join(DIRECTORY, 'GloVe', 'glove.6B.300d.txt'),encoding='utf-8') as file:
    for line in file:
        word, glove_vector = line.split(' ', maxsplit=1)
        glove_vector = np.array(glove_vector.split(' ')).astype('float')
        embedded_dictionary.update({word:glove_vector})

Helpers

In [82]:
def embed_sentence(sentences, embedded_dictionary, 
                       pos_tags = ['NN','NNS','NNP','NNPS','JJ','RB','VB',
                                      'VBG','VBN','VBP','VBZ','VBD']):

    sentences = sentences.replace('\n', ' ').replace('_', "").lower()
    tokens = nltk.word_tokenize(sentences)

    # tokens decide if a particular pos_tags is in the sentence or not.
    PoS = np.array([int(tag[1] in pos_tags) 
                         for tag in nltk.pos_tag(tokens)]).reshape((-1,1))

    # apply pos to the embeddings
    dictionary_dimensions = len(list(embedded_dictionary.values())[0])
    embedded_sentence = np.array([embedded_dictionary[token] 
                          if token in embedded_dictionary.keys() 
                          else np.zeros(dictionary_dimensions) 
                          for token in tokens])
    embedded_sentence *= PoS

    return embedded_sentence

def cos_dist(vec1, vec2):

    if np.sqrt(np.sum(vec1 ** 2)) == 0 or np.sqrt(np.sum(vec2 ** 2)) == 0:
        return 1
    else:
        return 1 - np.dot(vec1, vec2) / (np.sqrt(np.sum(vec1 ** 2)) * np.sqrt(np.sum(vec2 ** 2)))

def moving_window(search_content, search_query, increase_padding_by = 0):

    query_length = search_query.shape[0]
    padded_query_length = query_length + increase_padding_by

    # bag-of-words excerpt embedding
    excerpt_embedding = search_query.mean(axis=0)

    # window moving size
    current_window = np.array([search_content[line-padded_query_length:line].mean(axis=0) 
                            for line in range(padded_query_length, search_content.shape[0])])

    # sliding distance: distance between sliding window and excerpt embeddings
    window_query_cosdistance = np.array([cos_dist(current_window[line,:], 
                                         excerpt_embedding) 
                         for line in range(current_window.shape[0])])

    return window_query_cosdistance

def find_sentence(search_content, pos, len):

    search_content = search_content.replace('\n', ' ').replace('_', "").lower()
    closest_content = nltk.word_tokenize(search_content)
    closest_content = closest_content[pos : pos + len]
    closest_content = ' '.join(closest_content).replace(' ,',',').replace(' .','.')
#     print(closest_content)
    return closest_content

def connotative_search(excerpt, book, padding = 4):
    # embed search query
    book_embedding = embed_sentence(book, embedded_dictionary)
    excerpt_embedding = embed_sentence(excerpt, embedded_dictionary)
    excerpt_word_count = len(nltk.word_tokenize(excerpt))

    # calculate cosine distance/similarity
    distances = moving_window(book_embedding, 
                                 excerpt_embedding, 
                                 increase_padding_by = padding)

    # find minimum cosine distance and locate the respective window containing that sentence
    match = find_sentence(book, distances.argmin(), 
                       excerpt_word_count + padding)
    
    return match

In [24]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
nltk.download('averaged_perceptron_tagger')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Anirudh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

Here sample_text is the content you want the search window to read.

Search query and the search content is provided to the locate method
The automatic padding provided is by default 5 since it gave me the best results for the sample text.

In [84]:
query = ("But, wonderfully to tell, not an irregular, shapeless fragment of raw ore fresh from nature's crucible.")
# query = input("Enter your search query :-")
# query = query.replace('\n', ' ').replace('_', "").lower()
# Document = (input("Enter your search Document -"))
Document= ("sample_text.txt")
with open(Document,encoding='utf-8') as f:
    doc = str(f.readlines())

book_embedding = sequence_embedding(doc, embedded_dictionary)

Closest result:- \n ', `` but, wonderful to relate, not an irregular, shapeless fragment of crude ore, fresh from nature 's crucible


In [86]:
print("Closest result:-",connotative_search(query, doc)) 

Closest result:- \n ', `` but, wonderful to relate, not an irregular, shapeless fragment of crude ore, fresh from nature 's crucible
